In [35]:
import numpy as np
import pandas as pd
from scipy.special import factorial,multigammaln
from numpy.testing import assert_almost_equal

from sklearn.preprocessing import normalize
from sklearn import cluster
import time
import matplotlib.pyplot as plt

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import multiprocessing as mp
from multiprocessing import Pool, Value, Array

import numba
from numba import jit
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


# Glass Data

In [128]:
## Load the sample data from UCI on the element composition and type of glass for testing

columns=["Refractive_Index" , "Sodium" ,"Magnesium","Aluminum","Silicon","Potassium","Calcium","Barium","Iron","Type of glass"]
df=pd.read_csv("Data_glass.csv",header=None,names=columns)

df = df[df["Type of glass"].values > 3]
X=np.array(df.iloc[:, :-1])
y1star=np.array(df.iloc[:, -1])


In [129]:
glass_data =(X-X.mean(axis=0))/X.std(axis=0)

array([[-0.0906998 ,  0.66068783,  0.18514486,  0.14000306,  0.03710368,
        -0.02969467, -0.62753183, -0.07168545, -0.31653152],
       [-0.38483643,  0.28590885,  0.06733962,  0.3994663 ,  0.26300742,
        -0.22503748, -0.6271991 , -0.0673065 , -0.297196  ],
       [-0.12582806, -0.01912966,  0.61586613,  0.04586668, -0.11873649,
         0.28520817, -0.49332224, -0.11340276, -0.5007369 ],
       [-0.1847304 ,  0.06560335,  0.31991457, -0.06980579,  0.50550144,
         0.18544993, -0.6012942 , -0.09948709, -0.43929138],
       [-0.29702347, -0.27673928,  0.13449525,  0.3574314 ,  0.16917912,
         0.22236416, -0.28038234, -0.04639064,  0.72836289],
       [-0.18300299,  0.1076018 ,  0.25888764, -0.29208101,  0.52238435,
         0.28472218, -0.49264437, -0.1002376 , -0.4426053 ],
       [-0.12429982, -0.08719733,  0.25361537, -0.42907042,  0.62697607,
         0.22000741, -0.3607894 , -0.08747802, -0.38626462],
       [ 0.22935496,  0.71449469,  0.12343249,  0.13763798, -0

# Synthetic Gaussian Data

In [39]:
## Synthetic Gaussian Data with mean=0 and variance =1
mean=np.zeros(2)
var=np.eye(2)
synthetic=np.random.multivariate_normal(mean,var,size=20)

In [40]:
## Synthetic Gaussian Data with four seperate means (-1,1), (-1,-1), (1, -1), (1,1)

In [41]:
mean0 = np.array((-1,1))
mean1 = np.array((-1,-1)) 
mean2 = np.array((1,-1))
mean3 = np.array((1,1))

var=np.eye(2)

clustersyn0 = np.random.multivariate_normal(mean0,var,size=5)
clustersyn1 = np.random.multivariate_normal(mean1,var,size=5)
clustersyn2 = np.random.multivariate_normal(mean2,var,size=5)
clustersyn3 = np.random.multivariate_normal(mean3,var,size=5)

Synthetic1 = np.array((clustersyn0, clustersyn1, clustersyn2, clustersyn3)).reshape(20,2)
y2star = np.repeat(np.array([1,2,3,4]),5)

# Brain vs Head Size Data

In [42]:
columns=["Sex", "Age_range", "Head_size", "Brain_weight"]
df=pd.read_csv("Brain_Weight.csv", sep = "    " , header=None,names=columns)

df['Category'] = 0
df.loc[(df.Sex == 1) & (df.Age_range == 1), "Category"] = 0
df.loc[(df.Sex == 1) & (df.Age_range == 2), "Category"] = 1
df.loc[(df.Sex == 2) & (df.Age_range == 1), "Category"] = 2
df.loc[(df.Sex == 2) & (df.Age_range == 2), "Category"] = 3
cols = df.columns.tolist()
df1 = df[['Category', 'Head_size','Brain_weight']]

df1 = df1.sample(50)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [43]:
a1 = df1.iloc[:,1].values
a2 = df1.iloc[:,2].values
y3star = df1.iloc[:,0].values
df2 = np.c_[a1,a2]
brain_data = (df2-df2.mean(axis=0))/df2.std(axis=0)

# K-means and Agg. clustering performance times

In [47]:
%%time 
clust2=cluster.KMeans(n_clusters=4)
y_hat2=clust2.fit_predict(Synthetic1)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 27.8 ms


In [48]:
%%time 
clust2=cluster.KMeans(n_clusters=6)
y_hat2=clust2.fit_predict(glass_data)

CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 39.2 ms


In [49]:
%%time 
clust2=cluster.KMeans(n_clusters=4)
y_hat2=clust2.fit_predict(brain_data)

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 30.3 ms


In [55]:
%%time
clust=cluster.AgglomerativeClustering(n_clusters=4)
y_hat=clust.fit_predict(Synthetic1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 895 µs


In [56]:
%%time
clust=cluster.AgglomerativeClustering(n_clusters=6)
y_hat=clust.fit_predict(glass_data)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.54 ms


In [57]:
%%time
clust=cluster.AgglomerativeClustering(n_clusters=4)
y_hat=clust.fit_predict(brain_data)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.22 ms


# Algorithms

In [151]:
## Define a new class called note which has the data points d_k and the number of the cluster

class Node:
    from scipy.special import factorial

    
    def __init__(self,p,alpha,i):
        
        self.single=True
        self.points=set()
        self.points.add(p)
        self.d=alpha
        self.number=i
        self.left=None
        self.right=None
        self.ph=0
            
    def add(self,x):
        self.points.add(x)
        
    def add_all(self,x):
        self.points=x
        
    def remove(self,x):
        self.points.remove(x)
        
    def combine(self,y,alpha):
        p=self.points.union(y.points)
        z=Node(1,self.d,self.number)
        z.left=self
        z.right=y
        z.remove(1)
        z.points=p
        z.d= alpha*factorial(len(p)-1)+self.d*y.d
        z.single=False
        return z
## Function that initiates clusters
def init(X,alpha,kappa0,v0,mu0,eta0):
    x=[]
    for i in range(len(X)):
        node=Node(i,alpha,i)
        node.ph=prob_hypo(X[[i]],kappa0,v0,mu0,eta0)
        x.append(node)
        
    return x
        
def prob_hypo(X,kappa0,v0,mu0,eta0):
    from decimal import Decimal
    nf,df= X.shape
    n=Decimal(nf)
    d=Decimal(df)
    a= (1/(Decimal(np.pi))**(n*d/2))
    b=Decimal(multigammaln((v0+nf)/2,df)/multigammaln(v0/2,df))
    S=np.zeros((df,df))
    for i in range(nf):
        o=X[i]-X.mean(axis=0)
        S+=np.outer(o,o)
    etan=(eta0) + S + (kappa0*nf/(kappa0+nf))*np.outer((X.mean(axis=0)-mu0),(X.mean(axis=0)-mu0))
    c=Decimal(np.linalg.det(eta0)**(v0/2))/(Decimal(np.linalg.det(etan))**((Decimal(v0)+n)/2))
    d=Decimal(kappa0/(kappa0+nf))**(d/2)
    return float(a*b*c*d)

## Function that returns two matrices for each index combination
## param @rik : r values, param @pit, pi values
def calculate_r(nodes,alpha,X,kappa0,v0,mu0,eta0):
    from scipy.special import factorial
    n=len(nodes)
    rik=np.zeros((n,n))
    pit=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            clust_k=nodes[i].combine(nodes[j],alpha)
            nk=len(clust_k.points)
            dk=clust_k.d
            pi=alpha*factorial(nk-1)/dk
            all_points=list(clust_k.points)
            ph=prob_hypo(X[all_points][:],kappa0,v0,mu0,eta0) 
            pt=ph*pi+ (1-pi)*nodes[i].ph*nodes[j].ph
            pit[i,j]=pt
            rik[i,j]=(pi*ph)/pt
           
    np.fill_diagonal(rik,-10)
    return rik,pit

## Function for updating clusters, by creating a new node (combination of i and j) and deleting nodes i,j
def update_clust(rk,pit,nodes,alpha):
    i,j=np.unravel_index(np.argmax(rk),rk.shape)
    if len(nodes[i].points)>len(nodes[j].points):
        new_node=nodes[i].combine(nodes[j],alpha)
    else:
        new_node=nodes[j].combine(nodes[i],alpha)
    new_node.single=False
    new_node.ph=pit[i,j]
    nodes[i]=new_node 
    del nodes[j]
    return nodes
## Bayesian Cluster algorithm which returns the final classes only.
def bayesian_clust(X,alpha,kappa0,v0,mu0,eta0,k=3):
    nodes=init(X,alpha,kappa0,v0,mu0,eta0)
    
    for i in range(len(nodes)-k):
        rk,pit=calculate_r(nodes,alpha,X,kappa0,v0,mu0,eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
    y=np.zeros(len(X))
    for i in range(k):
        ind=list(nodes[i].points)
        y[ind]=i
    for i in range(k-1):
        rk,pit=calculate_r(nodes,alpha,X,kappa0,v0,mu0,eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
        
    return y,nodes[0]

In [152]:
## Making the loop faster-- Only loop to fill the upper triangular part of the numpy matrix
def calculate_r_fast(nodes,alpha,X,kappa0,v0,mu0,eta0):
    from scipy.special import factorial
    n=len(nodes)
    rik=np.zeros((n,n))
    pit=np.zeros((n,n))
    for i in range(n):
        for j in range(i+1,n):
            clust_k=nodes[i].combine(nodes[j],alpha)
            nk=len(clust_k.points)
            dk=clust_k.d
            pi=alpha*factorial(nk-1)/dk
            all_points=list(clust_k.points)
            ph=prob_hypo(X[all_points][:],kappa0,v0,mu0,eta0) 
            pt=ph*pi+ (1-pi)*nodes[i].ph*nodes[j].ph
            pit[i,j]=pt
            rik[i,j]=(pi*ph)/pt
           
    np.fill_diagonal(rik,-10)
    return rik,pit

## Bayesian Cluster algorithm which returns the final classes only.
def bayesian_clust_fast(X,alpha,kappa0,v0,mu0,eta0,k=3):
    nodes=init(X,alpha,kappa0,v0,mu0,eta0)
   
    for i in range(len(nodes)-k):
        rk,pit=calculate_r_fast(nodes,alpha,X,kappa0, v0,mu0, eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
    y=np.zeros(len(X))
    for i in range(k):
        ind=list(nodes[i].points)
        y[ind]=i
    for i in range(k-1):
        rk,pit=calculate_r_fast(nodes,alpha,X,kappa0,v0,mu0,eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
        
    return y,nodes[0]

In [155]:
%%cython -a
import numpy as np
import cython
from scipy.special import multigammaln
from decimal import Decimal
from cython.parallel import parallel, prange

##Recursive function that calculates factorial faster than scipy

@cython.wraparound(False)
@cython.boundscheck(False)
cdef fact(int x):
    if x==0 :
        return 1
    else:
        return x*fact(x-1)
    

class Node_cyt:
  
    def __init__(self,int p,float alpha,int i):
        self.single=True
        self.points=set()
        self.points.add(p)
        self.d=alpha
        self.number=i
        self.left=None
        self.right=None
        self.ph=0
            
    def add(self,int x):
        self.points.add(x)
        
    def add_all(self,int x):
        self.points=x
        
    def remove(self,int x):
        self.points.remove(x)
        
    def combine(self,y,float alpha):
        p=self.points.union(y.points)
        z=Node_cyt(1,self.d,self.number)
        z.left=self
        z.right=y
        z.remove(1)
        z.points=p
        z.d= alpha*fact(len(p)-1)+self.d*y.d
        z.single=False
        return z 

## Function that gives the indeces of maximum value within a Matrix


@cython.wraparound(False)
@cython.boundscheck(False)
cdef get_max(double[:,:] X):
    cdef int n,m,k,l,i,j
    cdef double arg,c
    k=X.shape[0]
    l=X.shape[1]
    arg=0.0
    with cython.nogil:
        for n in range(k):
            for m in range(l):
                c=X[n,m]
                if c>=arg:
                    arg=c
                    i=n
                    j=m
    return i,j

## Function that calculates the mean of a row
@cython.wraparound(False)
@cython.boundscheck(False)
cdef get_mean_1(double[:,:] X,double[:] res):
   
    cdef int i,j,k,m
    cdef double mean
    
    k=X.shape[0]
    m=X.shape[1]
    with cython.nogil ,parallel():
        for j in range(m):
            mean=0
            for i in range(k):
                mean+=X[i,j]
            mean=mean/k
            res[j]=mean
    return np.array(res)

@cython.wraparound(False)
@cython.boundscheck(False)

cdef outer_product(double[:] x, double[:,:] res):
    cdef int n,i,j
    n=x.shape[0]
    with cython.nogil, parallel():
        for i in range(n):
            for j in range(n):
                res[i,j]=x[i]*x[j]
                
@cython.wraparound(False)
@cython.boundscheck(False)
def prob_hypo_cyt( double[:,:] X,double kappa0,double v0, double[:] mu0, double[:,:] eta0):
    cdef int nf, df,i
    nf= X.shape[0]
    df=X.shape[1]
    res=np.zeros((df,df))
    n=Decimal(nf)
    d=Decimal(df)
    a=(1/(Decimal(np.pi))**(n*d/2))
    b=Decimal(multigammaln((v0+nf)/2,df)/multigammaln(v0/2,df))
    S=np.zeros((df,df))
    mean=np.zeros(df)
    mean=get_mean_1(X,mean)
    for i in range(nf):
        o=X[i]-mean
        outer_product(o,res)
        S+=res
    
    outer_product(mean-mu0,res)
    etan=(eta0) + S + (kappa0*nf/(kappa0+nf))*res
    c=Decimal(np.linalg.det(eta0)**(v0/2))/(Decimal(np.linalg.det(etan))**((Decimal(v0)+n)/2))
    d=Decimal(kappa0/(kappa0+nf))**(d/2)
    return float(a*b*c*d)
   

@cython.wraparound(False)
@cython.boundscheck(False)
def init_cyt(X,double alpha, double kappa0,double v0, double[:] mu0, double[:,:] eta0):  
    cdef int i,n
    n=X.shape[0]
    x=[]
    for i in range(n):
        node=Node_cyt(i,alpha,i)
        node.ph=prob_hypo_cyt(X[[i]],kappa0,v0,mu0,eta0)
        x.append(node)
    return x



@cython.wraparound(False)
@cython.boundscheck(False)
def calculate_r_cyt(nodes,double alpha,X,double kappa0,double v0,double[:] mu0, double[:,:] eta0):
    
    cdef int n,i,j,nk
    cdef float dk
    n=len(nodes)
    rik=np.zeros((n,n))-10
    pit=np.zeros((n,n))-10
    for i in range(n):
        for j in range(i+1,n):
            clust_k=nodes[i].combine(nodes[j],alpha)
            nk=len(clust_k.points)
            dk=clust_k.d
            pi=alpha*fact(nk-1)/dk
            all_points=list(clust_k.points)
            ph=prob_hypo_cyt(X[all_points][:],kappa0,v0,mu0,eta0) 
            pt=ph*pi+ (1-pi)*nodes[i].ph*nodes[j].ph
            pit[i,j]=pt
            rik[i,j]=(pi*ph)/pt
           
    
    return rik,pit

@cython.wraparound(False)
@cython.boundscheck(False)
cdef update_clust_cyt(double[:,:] rk,double[:,:] pit,nodes,double alpha):
    cdef int i,j,n,f
    indi=get_max(rk)
    i=indi[0]
    j=indi[1]
    if len(nodes[i].points)>len(nodes[j].points):
        new_node=nodes[i].combine(nodes[j],alpha)
    else:
        new_node=nodes[j].combine(nodes[i],alpha)
    new_node.single=False
    new_node.ph=pit[i,j]
    nodes[i]=new_node 
    del nodes[j]
    return nodes

@cython.wraparound(False)
@cython.boundscheck(False)
def bayesian_clust_cyt(X,double alpha,double kappa0,double v0,mu0,double[:,:] eta0,int k=3):
   
    nodes=init_cyt(X,alpha,kappa0,v0,mu0,eta0)
    cdef int n
    n=X.shape[0]
    rik=np.zeros((n,n))-10
    for i in range(n-k):
        rk,pit=calculate_r_cyt(nodes,alpha,X,kappa0, v0,mu0, eta0)
        nodes=update_clust_cyt(rk,pit,nodes,alpha)
    y=np.zeros(n)
    for i in range(k):
        ind=list(nodes[i].points)
        y[ind]=i
    for i in range(k-1):
        rk,pit=calculate_r_cyt(nodes,alpha,X,kappa0,v0,mu0,eta0)
        nodes=update_clust_cyt(rk,pit,nodes,alpha)
        
    return y,nodes[0]



In [10]:

## Function that gives all the nodes in a tree recursively
def get_allthenodes(tree,nodes):
    if tree==None:
        pass
    else:
        nodes.append(tree)
        get_allthenodes(tree.left,nodes)
        get_allthenodes(tree.right,nodes)
        
## Simple functions to keep track of points in the sets        
def contains(x,i,j):
    
    if i in x and j in x:
        return True
    else:
        return False
    
## Function that calculate the smallest subtree given the list of all nodes
def find_subtree(i,j,nodes):
    n=float('inf')
    for k in nodes :
        s=k.points
        if contains(s,i,j) and len(s)<n:
            res=k
            n=len(s)
    return res
        
def calc_impurity(tree,yhat,y,n=100):
    impurity=0
    for i in range(n):
        a=np.arange(len(y))
        i=np.random.choice(a)
        clust=y[i]
        a=np.where(y==clust)[0]
        j=np.random.choice(a)
        nodes=[]
        get_allthenodes(tree,nodes)
        smallest=find_subtree(i,j,nodes)
        s=0
        for i in smallest.points:
            if[y[i]]==clust:
                s+=1
        ratio=s/len(smallest.points)
        impurity+=ratio
    impurity=impurity/n
    return impurity

# Prior Analysis Synthetic Cluster Data

In [156]:
X=Synthetic1
eta0=np.cov(X, rowvar=False)
mu0=X.mean(axis=0)
kappa0=len(X)/20
v0=kappa0*2


In [157]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 404 ms, sys: 0 ns, total: 404 ms
Wall time: 407 ms


In [158]:
#Impurity Score
tree = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[1]
calc_impurity(tree,y3,y2star,n=100)

0.6293901098901099

In [159]:
kappa0=len(X)/20
v0=kappa0*5

In [160]:
%%time
#Bayesian
y1=bayesian_clust(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 1.7 s, sys: 56 ms, total: 1.76 s
Wall time: 1.75 s


In [162]:
%%time
#Fast Bayesian 
y2=bayesian_clust_fast(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 828 ms, sys: 56 ms, total: 884 ms
Wall time: 852 ms


In [163]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 660 ms, sys: 4 ms, total: 664 ms
Wall time: 667 ms


In [164]:
#Impurity Score
tree = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[1]
calc_impurity(tree,y3,y2star,n=100)

0.6091767676767675

In [40]:
kappa0=len(X)/40
v0=kappa0*4

In [42]:
#Impurity Score
tree = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[1]
calc_impurity(tree,y3,y2star,n=100)

0.5640714285714286

In [43]:
kappa0=len(X)/5
v0=kappa0

In [44]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 492 ms, sys: 0 ns, total: 492 ms
Wall time: 489 ms


In [45]:
#Impurity Score
tree = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[1]
calc_impurity(tree,y3,y2star,n=100)

0.6307619047619046

In [46]:
kappa0=len(X)/20
v0=kappa0*3

In [49]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 660 ms, sys: 4 ms, total: 664 ms
Wall time: 673 ms


In [50]:
#Impurity Score
tree = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[1]
calc_impurity(tree,y3,y2star,n=100)

0.6207142857142857

# Prior Analysis Glass Data

In [165]:
X=glass_data
eta0=np.cov(X, rowvar=False)
mu0=X.mean(axis=0)
kappa0=len(X)/20
v0=kappa0*5

In [166]:
%%time
#Bayesian
y1=bayesian_clust(X,3,kappa0,v0,mu0,eta0,k=3)[0]

CPU times: user 47.2 s, sys: 840 ms, total: 48.1 s
Wall time: 48 s


In [167]:
%%time
#Fast Bayesian 
tree, y2=bayesian_clust_fast(X,3,kappa0,v0,mu0,eta0,k=3)


CPU times: user 22.9 s, sys: 412 ms, total: 23.3 s
Wall time: 23.3 s


In [168]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,3,kappa0,v0,mu0,eta0,k=3)

CPU times: user 20.6 s, sys: 60 ms, total: 20.7 s
Wall time: 21 s


In [169]:
#Impurity Score
tree = bayesian_clust_cyt(X,3,kappa0,v0,mu0,eta0,k=3)[1]
calc_impurity(tree,y3,y1star,n=100)

0.6694036032378687

# Prior Analsysis of Brain Size vs Head Size Dataset

In [170]:
X=brain_data
eta0=np.cov(X, rowvar=False)
mu0=X.mean(axis=0)
kappa0=len(X)/20
v0=kappa0*2

In [171]:
%%time
#Bayesian
y1=bayesian_clust(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 25.9 s, sys: 672 ms, total: 26.6 s
Wall time: 26.5 s


In [172]:
%%time
#Fast Bayesian 
y2=bayesian_clust_fast(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 12.5 s, sys: 504 ms, total: 13 s
Wall time: 12.8 s


In [174]:
%%time
#Cythonized 
y3=bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 10.1 s, sys: 32 ms, total: 10.2 s
Wall time: 10.3 s


In [173]:
#Impurity Score
y3,tree = bayesian_clust_cyt(X,6,kappa0,v0,mu0,eta0,k=6)
calc_impurity(tree,y3,y3star,n=100)

0.46593168796869977

In [175]:
kappa0=len(X)/20
v0=kappa0*5

In [176]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 10.7 s, sys: 4 ms, total: 10.7 s
Wall time: 10.8 s


In [177]:
#Impurity Score
tree = bayesian_clust_cyt(X,6,kappa0,v0,mu0,eta0,k=6)[1]
calc_impurity(tree,y3,y3star,n=100)

0.476990476190476

In [178]:
kappa0=len(X)/40
v0=kappa0*4

In [179]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 9.74 s, sys: 20 ms, total: 9.76 s
Wall time: 9.8 s


In [180]:
#Impurity Score
tree = bayesian_clust_cyt(X,6,kappa0,v0,mu0,eta0,k=6)[1]
calc_impurity(tree,y3,y3star,n=100)

0.4133392098323132

In [181]:
kappa0=len(X)/5
v0=kappa0

In [182]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 5.98 s, sys: 24 ms, total: 6 s
Wall time: 6.04 s


In [183]:
#Impurity Score
tree = bayesian_clust_cyt(X,6,kappa0,v0,mu0,eta0,k=6)[1]
calc_impurity(tree,y3,y3star,n=100)

0.3629084513697517

In [184]:
kappa0=len(X)/20
v0=kappa0*3

In [185]:
%%time
#Cythonized 
y3 = bayesian_clust_cyt(X,4,kappa0,v0,mu0,eta0,k=4)[0]

CPU times: user 10.9 s, sys: 0 ns, total: 10.9 s
Wall time: 11 s


In [186]:
#Impurity Score
tree = bayesian_clust_cyt(X,6,kappa0,v0,mu0,eta0,k=6)[1]
calc_impurity(tree,y3,y3star,n=100)

0.46042777777777766